In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC
from creds import *
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import os
import numpy as np

In [2]:
def wait_and_click_element(selector, by=By.CSS_SELECTOR, sleep_for=3):
    try:
        time.sleep(sleep_for)
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((by, selector))
        )
        element.click()

    except Exception as e:
        print("Error: ", e)

def get_element(selector, by=By.ID, sleep_for=3):
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((by, selector))
        )
        time.sleep(sleep_for)
        return element

    except Exception as e:
        print("Error: ", e)

def get_all_elements(selector, by=By.ID, sleep_for=3):
    try:
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((by, selector))
        )
        time.sleep(sleep_for)
        return elements

    except Exception as e:
        print("Error: ", e)

def set_value_to_element(element, value, sleep_for=3):
    element.clear()
    time.sleep(sleep_for)
    element.clear()
    try:
        element.send_keys(value)
    except Exception as e:
        print("Error: ", e)

def let_the_page_load():
    while driver.execute_script("return document.readyState;") != "complete":
        time.sleep(1)

def wait_for_a_bit(value=5):
    time.sleep(value)

In [3]:
# start chrome driver and go to lexis nexis site on ubc website
driver = webdriver.Chrome()
driver.get('https://resources.library.ubc.ca/page.php?id=2472')

In [4]:
# hit button to go to cwl
let_the_page_load()
wait_and_click_element("a.green.medium.button")

In [5]:
# set username and password for cwl
let_the_page_load()
username = get_element("username", sleep_for=1)
password = get_element("password", sleep_for=1)
set_value_to_element(username, uname, sleep_for=1)
set_value_to_element(password, pwd, sleep_for=1)
wait_and_click_element("_eventId_proceed", By.NAME)

In [6]:
# weirdly, i have to reload the page for it to work
wait_for_a_bit(20)
let_the_page_load()
driver.refresh()

## Search keyword and date range modification

In [7]:
search = "community"

In [9]:
let_the_page_load()
search_keyword = get_element("lng-expanding-textarea[contenteditable='true']", By.CSS_SELECTOR, sleep_for=0)
set_value_to_element(search_keyword, search, sleep_for=0)
wait_and_click_element('//button[@aria-label="Search"]', By.XPATH)

## Setting the filters

In [14]:
# clearing the filters
let_the_page_load()
wait_and_click_element("button.clear-filters[data-action='clear']", By.CSS_SELECTOR)

In [16]:
new_min_date = "01/01/1966"
new_max_date = "12/31/1971"
wait_for_a_bit()

In [17]:
timeline_element = get_element("podfiltersbuttondatestr-news", By.ID)
timeline_element.click()
wait_for_a_bit()

In [18]:
timeline_element = get_element("podfiltersbuttondatestr-news", By.ID)
timeline_element.send_keys(Keys.TAB, Keys.TAB, Keys.TAB, new_min_date, Keys.TAB, Keys.TAB, new_max_date)
ok_button = wait_and_click_element(".//button[contains(@class, 'save')]", By.XPATH)
wait_for_a_bit()

---

In [13]:
wait_and_click_element("podfiltersbuttonpublicationtype", By.ID)
wait_for_a_bit()
wait_and_click_element(selector='ul[data-id="publicationtype"] li.sel-multi button[data-action="selmulti"]', by=By.CSS_SELECTOR)

In [13]:
wait_and_click_element("#podfiltersbuttonen-geography-news")
wait_for_a_bit()
wait_and_click_element("label[for='_tmnyk_pf51']")
wait_for_a_bit(10)

In [14]:
wait_and_click_element("#podfiltersbuttonlanguage")
wait_for_a_bit()
wait_and_click_element("label[for='_tmnyk_pf52']", By.CSS_SELECTOR)
wait_for_a_bit(13)

In [15]:
wait_and_click_element("#podfiltersbuttonsource")
wait_for_a_bit()
wait_and_click_element(selector='ul[data-id="source"] li.sel-multi button[data-action="selmulti"]', by=By.CSS_SELECTOR)

In [16]:
source_names_to_select = ["New York Times", "Guardian"]

let_the_page_load()

for source_name in source_names_to_select:
    try:
        xpath = "//li[label[contains(.,'" + source_name + "')]]/label/input[@type='checkbox']"
        wait_and_click_element(xpath, By.XPATH)
    except Exception as e:
        print(f"Error with {source_name}: ", e)

wait_and_click_element(selector=".button.primary.saveaddmultifilter", by=By.CSS_SELECTOR)
wait_for_a_bit(10)

## Extract articles on each page

In [14]:
PAGE = 200
for i in range(PAGE):
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Next']"))
        )
        next_page_button.click()
    except:
        print('could not go to next page ', i)

could not go to next page  1
could not go to next page  2
could not go to next page  4
could not go to next page  5
could not go to next page  7
could not go to next page  9
could not go to next page  10
could not go to next page  12
could not go to next page  13
could not go to next page  15
could not go to next page  16
could not go to next page  18
could not go to next page  19
could not go to next page  20
could not go to next page  22
could not go to next page  23
could not go to next page  25
could not go to next page  26
could not go to next page  28
could not go to next page  29
could not go to next page  32
could not go to next page  34
could not go to next page  36
could not go to next page  37
could not go to next page  39
could not go to next page  41
could not go to next page  42
could not go to next page  44
could not go to next page  46
could not go to next page  48
could not go to next page  49
could not go to next page  51
could not go to next page  53
could not go to 

In [21]:
index = 1
page = 1
csv_file_path = '../data/data_1972_to_1979.csv'

In [22]:
while True:
    article_xpaths = [f'/html/body/div[1]/div/main/div[2]/div/div[2]/div[2]/form/div[2]/ol/li[{i}]/div/h2/a' for i in range(0, 21, 2)]
    
    for xpath in article_xpaths:
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath))).click()
        except Exception as e:
            print(f"page: {page}, article: {index}")
            index += 1
            print('cannot click this path')
            # print(xpath)
            continue
        try:   
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "SS_DocumentTitle"))
            ).text
            doc_info = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "SS_DocumentInfo"))
            )
            source, date = doc_info[0].text, doc_info[1].text
            body_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "SS_LeftAlign"))
            )
            
            body = ''.join([p.text for p in body_elements.find_elements(By.CSS_SELECTOR, "p")[1:]])
            data = pd.DataFrame([[title, source, date, body]], columns=['Title', 'Source', 'Date', 'Body'])
            include_header = not os.path.exists(csv_file_path)
            data.to_csv(csv_file_path, mode='a', header=include_header, index=False)
            
            print(f"page: {page}, article: {index}")
            index += 1
            print(date, title)
            
            driver.back()  
        except Exception as e:
            print(f"page: {page}, article: {index}")
            index += 1
            print("Error with xpath")    

            driver.back()
    
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Next']"))
        )
        next_page_button.click()
        index = 0
        page += 1
    except Exception as e:
        print("No more pages or encountered an error navigating to the next page:", e)
        break

page: 1, article: 1
cannot click this path
page: 1, article: 2
cannot click this path
page: 1, article: 3
January 31, 1979, Wednesday The Community Reinvestment Act and Regulation BB
page: 1, article: 4
September 25, 1979, Tuesday C&S Community Loan Unit Set
page: 1, article: 5
December 4, 1979, Tuesday Community Bank Seminar Slated
page: 1, article: 6
April 26, 1979, Thursday Community Act Little Used, Heimann Says
page: 1, article: 7
January 19, 1979, Friday S&Ls Get $187 Mil. Community Funds
page: 1, article: 8
July 25, 1979, Wednesday Community Service Has Roots in Bank's History
page: 1, article: 9
February 9, 1979, Friday Community Banker Reserve Plan Receives Mixed Reaction
page: 1, article: 10
December 24, 1979, Monday Fed to Stress Community Dialogue In Reviewing Bank Expansion Bids
page: 1, article: 11
December 14, 1979, Friday European Community Pursues Regional Approach to LDC Aid;
European Economic community
page: 2, article: 0
cannot click this path
page: 2, article: 1
Ma

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=122.0.6261.94)
Stacktrace:
0   chromedriver                        0x00000001010c453c chromedriver + 3966268
1   chromedriver                        0x00000001010bcac8 chromedriver + 3934920
2   chromedriver                        0x0000000100d3fda0 chromedriver + 277920
3   chromedriver                        0x0000000100d28c74 chromedriver + 183412
4   chromedriver                        0x0000000100d28ba0 chromedriver + 183200
5   chromedriver                        0x0000000100dba320 chromedriver + 779040
6   chromedriver                        0x0000000100d76fb0 chromedriver + 503728
7   chromedriver                        0x0000000100d77a28 chromedriver + 506408
8   chromedriver                        0x0000000101089724 chromedriver + 3725092
9   chromedriver                        0x000000010108dc18 chromed

In [17]:
# driver.quit()